In [4]:
import tensorflow as tf
import torch
import numpy as np
from tqdm import tqdm
tf.config.list_physical_devices()
# tf.config.run_functions_eagerly(run_eagerly)
# tf.__version__

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
translate_template_de_en = lambda x:  "Translate '%s' from german to english"%x

In [6]:
import pandas as pd
# df = pd.read_csv("https://raw.githubusercontent.com/Shivanandroy/T5-Finetuning-PyTorch/main/data/news_summary.csv")
# WMT16 dataset
from datasets import load_dataset, DownloadMode, load_metric

train_df = load_dataset("wmt16", "de-en", split="train")
# eval_df = load_dataset("wmt16", "de-en", split="validation")



# eval_df_ds = pd.DataFrame({
#     "source_text":[i['translation']['en'] for i in eval_df],
#     "target_text":[i['translation']['de'] for i in eval_df]
# })
# eval_df_ds = eval_df_ds.applymap(str)

Found cached dataset wmt16 (/home/agupt135/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)
100%|█| 4548885/4548885 [03:30<00:00, 21583.84it/s]


In [11]:
# Create data frame
df = pd.DataFrame({
    "input_text":[translate_template_de_en(train_df[i]['translation']['de']) for i in tqdm(range(len(train_df))) ],
    "target_text":[train_df[i]['translation']['de'] for i in tqdm(range(len(train_df))) ],
    "prefix":np.array(["bart"]*len(train_df))
}).applymap(str)

100%|█| 4548885/4548885 [03:34<00:00, 21208.42it/s]


In [12]:
# Import created dataset
# df = pd.read_csv("task.csv")
# df = df.iloc[:,2:]
df.head()

,input_text,target_text,prefix
0,Translate 'Wiederaufnahme der Sitzungsperiode'...,Wiederaufnahme der Sitzungsperiode,bart
1,"Translate 'Ich erkläre die am Freitag, dem 17....","Ich erkläre die am Freitag, dem 17. Dezember u...",bart
2,"Translate 'Wie Sie feststellen konnten, ist de...","Wie Sie feststellen konnten, ist der gefürchte...",bart
3,Translate 'Im Parlament besteht der Wunsch nac...,Im Parlament besteht der Wunsch nach einer Aus...,bart
4,Translate 'Heute möchte ich Sie bitten - das i...,Heute möchte ich Sie bitten - das ist auch der...,bart


In [27]:
print(df.get("input_text")[0],"\n",df.get("target_text")[0],"\n",df.get("input_text")[1],"\n",df.get("target_text")[1])

Translate 'Wiederaufnahme der Sitzungsperiode' from german to english 
 Wiederaufnahme der Sitzungsperiode 
 Translate 'Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.' from german to english 
 Ich erkläre die am Freitag, dem 17. Dezember unterbrochene Sitzungsperiode des Europäischen Parlaments für wiederaufgenommen, wünsche Ihnen nochmals alles Gute zum Jahreswechsel und hoffe, daß Sie schöne Ferien hatten.


In [58]:
# WMT16 dataset
from datasets import load_dataset
from datasets import DownloadMode

train_df = load_dataset("wmt16", "de-en", split="train")
eval_df = load_dataset("wmt16", "de-en", split="validation")

Found cached dataset wmt16 (/home/agupt135/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)
Found cached dataset wmt16 (/home/agupt135/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)


{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'}}

In [59]:
eval_df[0]

{'translation': {'de': 'Die Premierminister Indiens und Japans trafen sich in Tokio.',
  'en': 'India and Japan prime ministers meet in Tokyo'}}

In [60]:
train_df[0]

{'translation': {'de': 'Wiederaufnahme der Sitzungsperiode',
  'en': 'Resumption of the session'}}

In [61]:
# Create data frame
train_df_ds = pd.DataFrame({
    "input_text":[i['translation']['en'] for i in train_df],
    "target_text":[i['translation']['de'] for i in train_df],
    
})

eval_df_ds = pd.DataFrame({
    "input_text":[i['translation']['en'] for i in eval_df],
    "target_text":[i['translation']['de'] for i in eval_df]
})
train_df_ds = train_df_ds.applymap(str)
eval_df_ds = eval_df_ds.applymap(str)
train_df_ds(head)

,input_text,target_text
0,Resumption of the session,Wiederaufnahme der Sitzungsperiode
1,I declare resumed the session of the European ...,"Ich erkläre die am Freitag, dem 17. Dezember u..."
2,"Although, as you will have seen, the dreaded '...","Wie Sie feststellen konnten, ist der gefürchte..."
3,You have requested a debate on this subject in...,Im Parlament besteht der Wunsch nach einer Aus...
4,"In the meantime, I should like to observe a mi...",Heute möchte ich Sie bitten - das ist auch der...
...,...,...
4548880,Their achievement remains one of the greatest ...,Das bleibt eine der größten Errungenschaften i...
4548881,"At the same time, Zuma’s revolutionary generat...",Gleichzeitig scheint sich Zumas revolutionäre ...
4548882,"In a region that reveres the elderly, Zuma’s a...","In einer Region, wo die älteren Menschen sehr ..."
4548883,Three in ten South Africans are younger than 1...,Drei von zehn Südafrikanern sind jünger als 15...


In [27]:
# BoolQ dataset
from datasets import load_dataset
from datasets import DownloadMode

train_df = load_dataset("boolq", split="train")
eval_df = load_dataset("boolq", split="validation")

train_df_ds = pd.DataFrame({
    "input_text":[i['question'] for i in train_df],
    "target_text":[i['answer'] for i in train_df],
    "prefix":np.array(["binary classification"]*len(train_df))
})

eval_df_ds = pd.DataFrame({
    "input_text":[i['question'] for i in eval_df],
    "target_text":[i['answer'] for i in eval_df],
    "prefix":np.array(["binary classification"]*len(eval_df)),
})
train_df_ds = train_df_ds.applymap(str)
eval_df_ds = eval_df_ds.applymap(str)
train_df_ds

,input_text,target_text,prefix
0,do iran and afghanistan speak the same language,True,binary classification
1,do good samaritan laws protect those who help ...,True,binary classification
2,is windows movie maker part of windows essentials,True,binary classification
3,is confectionary sugar the same as powdered sugar,True,binary classification
4,is elder scrolls online the same as skyrim,False,binary classification
...,...,...,...
9422,is a us district court a federal court,True,binary classification
9423,can a tenant get a restraining order against a...,True,binary classification
9424,is the golden state warriors in the playoffs,True,binary classification
9425,downton abbey will there be a season 7,False,binary classification


In [ ]:
# WMT16 dataset
dataset_url_training = "https://drive.google.com/file/d/1dWiHqnwNuCChBkSkG6_HAoJIGrOPC-4R/view?usp=share_link" 
data_dir_training = tf.keras.utils.get_file('flower_photos', origin=dataset_url_training, untar=True)
data_dir_training = pathlib.Path(data_dir_training)

In [15]:
for i in train_df:
    print(i)
    break
type(train_df)
train_df[0]

{'question': 'do iran and afghanistan speak the same language', 'answer': True, 'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.'}


{'question': 'do iran and afghanistan speak the same language',
 'answer': True,
 'passage': 'Persian (/ˈpɜːrʒən, -ʃən/), also known by its endonym Farsi (فارسی fārsi (fɒːɾˈsiː) ( listen)), is one of the Western Iranian languages within the Indo-Iranian branch of the Indo-European language family. It is primarily spoken in Iran, Afghanistan (officially known as Dari since 1958), and Tajikistan (officially known as Tajiki since the Soviet era), and some other regions which historically were Persianate societies and considered part of Greater Iran. It is written in the Persian alphabet, a modified variant of the Arabic script, which itself evolved from the Aramaic alphabet.'}

In [29]:
# Model building
import pandas as pd

from simpletransformers.t5 import T5Model
# from transformers import T5Tokenizer, TFT5Model

#     "evaluate_during_training": True,
#     "evaluate_during_training_steps": 15000,
#     "evaluate_during_training_verbose": True,

model_args = {
    "reprocess_input_data": True,
    "overwrite_output_dir": True,
    "max_seq_length": 128,
    "train_batch_size": 8,
    "num_train_epochs": 5,
    "save_eval_checkpoints": True,
    "save_steps": -1,
    "use_multiprocessing": True,
    "fp16": True,
    "use_cuda": True,
    "wandb_project": "BART",
}

model = T5Model("t5", "t5-small", args=model_args)

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print ("device ",device)
# model = model.to(device)

model.train(df, output_dir=None)

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/568611 [00:00<?, ?it/s]

KeyError: 4071352

Exception in thread ChkStopThr:
Traceback (most recent call last):
  File "/home/agupt135/anaconda3/lib/python3.9/threading.py", line 980, in _bootstrap_inner
    self.run()
  File "/home/agupt135/anaconda3/lib/python3.9/threading.py", line 917, in run
    self._target(*self._args, **self._kwargs)
  File "/home/agupt135/anaconda3/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 276, in check_stop_status
    self._loop_check_status(
  File "/home/agupt135/anaconda3/lib/python3.9/site-packages/wandb/sdk/wandb_run.py", line 214, in _loop_check_status
    local_handle = request()
  File "/home/agupt135/anaconda3/lib/python3.9/site-packages/wandb/sdk/interface/interface.py", line 787, in deliver_stop_status
    return self._deliver_stop_status(status)
  File "/home/agupt135/anaconda3/lib/python3.9/site-packages/wandb/sdk/interface/interface_shared.py", line 585, in _deliver_stop_status
    return self._deliver_record(record)
  File "/home/agupt135/anaconda3/lib/python3.9/site-packa

In [ ]:
# Calculate BLEU score
from nltk.translate.bleu_score import sentence_bleu
reference = [
    'this is a dog'.split(),
    'it is dog'.split(),
    'dog it is'.split(),
    'a dog, it is'.split() 
]
candidate = 'it is dog'.split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate )))

candidate = 'it is a dog'.split()
print('BLEU score -> {}'.format(sentence_bleu(reference, candidate)))

In [47]:
# Model prediction

to_predict = [
    "binary classification: do iran and afghanistan speak the same language",
    "binary classification: downton abbey will there be a season 7",
]
to_predict = tokenizer(to_predict)
# input_ids = tokenizer("translate English to German: The house is wonderful.", return_tensors="pt").input_ids

# outputs = model.generate(input_ids)
# print(tokenizer.decode(outputs[0], skip_special_tokens=True))

predictions = model.predict(to_predict)
print(predictions)

NameError: name 'tokenizer' is not defined

In [50]:
!pip install torch
!pip install transformers
!pip install pytorch_lightning

In [52]:
import time
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer, TFT5Model


def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# model = T5ForConditionalGeneration.from_pretrained('ramsrigouthamg/t5_boolean_questions')
model = TFT5Model.from_pretrained('t5-small')
tokenizer = T5Tokenizer.from_pretrained('t5-small')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print ("device ",device)
model = model.to(device)


def greedy_decoding (inp_ids,attn_mask):
  greedy_output = model.generate(input_ids=inp_ids, attention_mask=attn_mask, max_length=256)
  Question =  tokenizer.decode(greedy_output[0], skip_special_tokens=True,clean_up_tokenization_spaces=True)
  return Question.strip().capitalize()


def beam_search_decoding (inp_ids,attn_mask):
  beam_output = model.generate(input_ids=inp_ids,
                                 attention_mask=attn_mask,
                                 max_length=256,
                               num_beams=10,
                               num_return_sequences=3,
                               no_repeat_ngram_size=2,
                               early_stopping=True
                               )
  Questions = [tokenizer.decode(out, skip_special_tokens=True, clean_up_tokenization_spaces=True) for out in
               beam_output]
  return [Question.strip().capitalize() for Question in Questions]


def topkp_decoding (inp_ids,attn_mask):
  topkp_output = model.generate(input_ids=inp_ids,
                                 attention_mask=attn_mask,
                                 max_length=256,
                               do_sample=True,
                               top_k=40,
                               top_p=0.80,
                               num_return_sequences=3,
                                no_repeat_ngram_size=2,
                                early_stopping=True
                               )
  Questions = [tokenizer.decode(out, skip_special_tokens=True,clean_up_tokenization_spaces=True) for out in topkp_output]
  return [Question.strip().capitalize() for Question in Questions]

start = time.time()

# passage = "Starlink, of SpaceX, is a satellite constellation project being developed by Elon Musk and team to give satellite Internet go-to access for people in any part of the world. The plan is to comprise thousands of mass-delivered little satellites in low Earth circle, orbit, working in mix with ground handheld devices, for instance, our iPhones. Elon Musk speaks about it as a grand Idea that could change the way we view and access the world around us."
# truefalse ="yes"

# passage ="About 400 years ago, a battle was unfolding about the nature of the Universe. For millennia, astronomers had accurately described the orbits of the planets using a geocentric model, where the Earth was stationary and all the other objects orbited around it."
# truefalse ="yes"

passage ='''Months earlier, Coca-Cola had begun “Project Kansas.” It sounds like a nuclear experiment but it was just a testing project for the new flavor. In individual surveys, they’d found that more than 75% of respondents loved the taste, 15% were indifferent, and 10% had a strong aversion to the taste to the point that they were angry.'''
truefalse ="yes"

# passage ="The US has passed the peak on new coronavirus cases, President Donald Trump said and predicted that some states would reopen this month. The US has over 637000 confirmed Covid-19 cases and over 30826 deaths, the highest for any country in the world."
# truefalse = "yes"


text = "truefalse: %s passage: %s </s>" % (passage, truefalse)


max_len = 256

encoding = tokenizer.encode_plus(text, return_tensors="pt")
input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)



print ("Context: ",passage)
# print ("\nGenerated Question: ",truefalse)

# output = greedy_decoding(input_ids,attention_masks)
# print ("\nGreedy decoding:: ",output)

output = beam_search_decoding(input_ids,attention_masks)
print ("\nBeam decoding [Most accurate questions] ::\n")
for out in output:
    print(out)


output = topkp_decoding(input_ids,attention_masks)
print ("\nTopKP decoding [Not very accurate but more variety in questions] ::\n")
for out in output:
    print (out)

end = time.time()
print ("\nTime elapsed ", end-start)
print ("\n")


/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:163: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
/home/agupt135/anaconda3/lib/python3.9/site-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings

Context:  Months earlier, Coca-Cola had begun “Project Kansas.” It sounds like a nuclear experiment but it was just a testing project for the new flavor. In individual surveys, they’d found that more than 75% of respondents loved the taste, 15% were indifferent, and 10% had a strong aversion to the taste to the point that they were angry.

Beam decoding [Most accurate questions] ::

Does coca cola have a kansas flavor?
Is project kansas the same as coca cola?
Is project kansas a new coca cola flavor?


AttributeError: 'Tensor' object has no attribute 'tile'

In [53]:
!pip install -q pyspark==3.1.2 spark-nlp==4.2.8

In [54]:
import pandas as pd
import numpy as np
import json

from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp.base import *
from sparknlp.pretrained import PretrainedPipeline
from pyspark.sql.types import StringType, IntegerType

In [55]:
spark = sparknlp.start()
print ("Spark NLP Version :", sparknlp.version())
spark

JAVA_HOME is not set


Exception: Java gateway process exited before sending its port number

In [63]:
# document_assembler = DocumentAssembler()\
# .setInputCol("text")\
# .setOutputCol("documents")

t5 = T5Transformer() \
  .pretrained("t5_small", 'en') \
  .setTask("wmt1")\
  .setMaxOutputLength(200)\
  .setInputCols(train_df_ds.input_text) \
  .setOutputCol(train_df_ds.target_text)
# document_assembler, 
# pipe_components = [t5]
# pipeline = Pipeline().setStages(pipe_components)

# sentences = [['Anna and Mike is going skiing and they is liked is'],['Anna and Mike like to dance']]
# # df = spark.createDataFrame(sentences).toDF("text")
# df = train_df_ds

model1 = t5.fit(df)
annotated_df = model1.transform(df)
annotated_df.select(['text','t5.result']).show(truncate=False)

AttributeError: Cannot load _jvm from SparkContext. Is SparkContext initialized?